In [1]:
## Dans ce notebook, nous allons générer les données de la table delivery

In [2]:
# importation des packages
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
from unidecode import unidecode
import warnings 
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows", None)

In [3]:
# chargement des fichiers pdf dont on a besoin dans des dataframes python
df_geo = pd.read_csv("dataset/geographies.csv")
df_orders = pd.read_csv("dataset/orders.csv")
df_stores = pd.read_csv("dataset/stores.csv")
df_stores = df_stores.reset_index(drop=True)
df_emp = pd.read_csv("dataset/employees.csv")
df_shipping_cost = pd.read_csv("dataset/shipping_cost.csv")

In [4]:
lst_columns = ["Delivery_ID", "Order_ID", "Emp_ID", "Delivery_date"] # list colonne de notre dataframe

In [5]:
def search_store( df_geo, delivery_address , df_stores) :
    info_geo = df_geo[df_geo["District"] == delivery_address] # l'info dans la table geo
    department = info_geo["Department"].iloc[0] # le departement du district
    
    
    stores = df_stores[df_stores["Store_department"] == department] # les magasins avec ce département
    if not stores.empty :
        store_district = stores["Store_district"].tolist()
        if (delivery_address in store_district) :
            found_store = df_stores[ df_stores["Store_district"] == delivery_address]
            store_id = found_store["Store_ID"].iloc[0]
            
        else :
            
            delivery_address = random.choice(store_district)
            found_store = df_stores[ df_stores["Store_district"] == delivery_address]
            store_id = found_store["Store_ID"].iloc[0]
            
    else :
        number_list = [i for i in range(len(df_stores))]
        random_number = random.choice([0,1,2])

        found_store = df_stores.iloc[random_number]
        store_id = found_store["Store_ID"]
    return store_id

In [6]:
def delivery_man (df_emp, store_id):
    employe = df_emp[ (df_emp["Store_ID"] == store_id) & (df_emp["Emp_title"] == "Delivery Person")]
    number_list = [i for i in range(len(employe))]
    indice = random.choice(number_list)
    employe = employe.iloc[indice]
    employe_id = employe.iloc[0]
    return employe_id

In [ ]:
# génération de nos données
len_df_orders = len(df_orders)
lst_delivery = []
for i in range(1,len_df_orders+1):
    dic_o = {}
    dic_o["Delivery_ID"] = f"L{i:06d}"
    dic_o["Order_ID"] = df_orders["Order_ID"].iloc[i-1]
    delivery_address = df_orders["delivery_address"].iloc[i-1]
    store_id = search_store( df_geo, delivery_address , df_stores)
    dic_o["Emp_ID"] = delivery_man (df_emp, store_id)
    dic_o["Delivery_date"] = df_orders["Delivery_date"].iloc[i-1]
    lst_delivery.append(dic_o)


df_delivery = pd.DataFrame(columns= lst_columns)
for d in lst_delivery:
    df_delivery = df_delivery.append(d, ignore_index=True)

# enregistrement dans un fichier csv
df_delivery.to_csv("dataset/delivery.csv",encoding='utf-8-sig', index=False)

In [ ]:
df_delivery.head()

In [ ]:
len(df_delivery)